In [ ]:
import sys

In [ ]:
sys.path.append('/content/drive/MyDrive/BreastCancer')

In [ ]:
%pip install opendatasets --quiet
%pip install opencv-python --quiet
%pip install optuna --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.4 MB/s eta 0:00:00


In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/BreastCancer']

In [ ]:
import os
DIR = os.getcwd()
DIR

'/content'

In [ ]:
#import pandas as pd
import os
import opendatasets as od
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import optuna
from optuna.trial import TrialState
import torch
import numpy as np
import matplotlib.pyplot as plt

#import helpfunctions
from helpFunctions import myOrder, groupFiles, readFilesRGB, get_mean_std, train, objective

#import Breast Data Set class BreastCNN and TRAIN
from BreastDataSet import BreastDataSet
from MyResnet import MyResnet, preprocess

#DEVICE = torch.device("cpu")
BATCHSIZE = 16
CLASSES = 3
DIR = sys.path[-1]
EPOCHS = 100
LR = 0.001

#kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset

#Download dataset from kaggle
# if not os.path.exists('./breast-ultrasound-images-dataset'):
#     dataset = "https://www.kaggle.com/datasets/aryashah2k/breast-ultrasound-images-dataset"

#     od.download(dataset)

data_dir = DIR + "/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"

#benign data
benign_dir = data_dir + "/benign"
benign_files = [benign_dir + "/" + f for f in os.listdir(benign_dir) if "mask" not in f]
benign_files.sort(key=myOrder)
#grouped_bfiles = groupFiles(benign_files)
labels = [1]*len(benign_files)


#malignant data
malignant_dir = data_dir + "/malignant"
malignant_files = [malignant_dir + "/" + f for f in os.listdir(malignant_dir) if "mask" not in f]
malignant_files.sort(key=myOrder)
#grouped_mfiles = groupFiles(malignant_files)
labels_m = [2]*len(malignant_files)


#normal data
normal_dir = data_dir + "/normal"
normal_files = [normal_dir + "/" + f for f in os.listdir(normal_dir) if "mask" not in f]
normal_files.sort(key=myOrder)
#grouped_nrmfiles = groupFiles(normal_files)
labels_n = [0]*len(normal_files)

#Create list of all files
breast_files = benign_files
breast_files.extend(malignant_files)
breast_files.extend(normal_files)

#labels
labels.extend(labels_m)
labels.extend(labels_n)

breast_files_train, breast_files_test, labels_train, labels_test = train_test_split(breast_files, labels, stratify=labels,
                                                                                    test_size=0.15, shuffle=True)

breast_files_train, breast_files_valid, labels_train, labels_valid = train_test_split(breast_files_train, labels_train, stratify=labels_train,
                                                                                    test_size=0.2, shuffle=True)


breast_images_train = readFilesRGB(breast_files_train)
breast_images_valid = readFilesRGB(breast_files_valid)
breast_images_test = readFilesRGB(breast_files_test)

print(len(breast_images_train))

# #Data Augmentation for Training set
train_transform = v2.Compose([
    v2.Resize([256,256]),
    v2.RandomCrop(256, padding=8),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(180),
    v2.RandomVerticalFlip(),
    # v2.Resize([227,227]),
    # v2.ToImage(),
    # v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize(mean=mean_train, std=std_train)
])


train_dataset = BreastDataSet(breast_images_train, labels_train, transform=preprocess)
valid_dataset = BreastDataSet(breast_images_valid, labels_valid, transform=preprocess)
test_dataset = BreastDataSet(breast_images_test, labels_test, transform=preprocess)

if __name__ == "__main__":
    batch_size = BATCHSIZE
    num_epochs = EPOCHS
    learning_rate = LR #small batch sizes require small learning rates

    train_dl = DataLoader(train_dataset, batch_size, shuffle=True, drop_last=True)
    valid_dl = DataLoader(valid_dataset, batch_size, shuffle=True, drop_last=True)
    test_dl = DataLoader(test_dataset, batch_size, shuffle=True)

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(MyResnet(trial), 60, train_dl, valid_dl, trial), n_trials=200, timeout=7200)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


    model = MyResnet(trial)

    hist = train(model, num_epochs, train_dl, valid_dl, trial)

    torch.save(model.state_dict(), DIR + '/saved_model/breastCancerDetection.pth')

    model = MyResnet(trial)
    model.load_state_dict(torch.load(DIR + "/saved_model/breastCancerDetection.pth"))
    accuracy_test = 0
    model.eval()
    with torch.no_grad():
        for imgs_batch, labels_batch in test_dl:
            pred = model(imgs_batch)
            is_correct = (torch.argmax(pred, dim=1) == labels_batch).float()
            accuracy_test += is_correct.sum()
    accuracy_test /= len(test_dl.dataset)
    print(f'Test accuracy: {accuracy_test:.4f}')

    x_arr = np.arange(len(hist[0])) + 1
    fig = plt.figure(figsize=(12,4))
    ax = fig.add_subplot(1,2,1)
    ax.plot(x_arr, hist[0], "-o", label = "Train acc.")
    ax.plot(x_arr, hist[1], "--<", label = "Validation acc.")
    ax.legend(fontsize=15)
    ax.set_xlabel("Epoch", size=15)
    ax.set_ylabel("Loss", size=15)
    ax = fig.add_subplot(1,2,2)
    ax.plot(x_arr, hist[2], "-o", label = "Train loss")
    ax.plot(x_arr, hist[3], "-o", label = "Train loss")
    ax.legend(fontsize=15)
    ax.set_xlabel("Epoch", size=15)
    ax.set_ylabel("Accuracy", size=15)
    fig.savefig(DIR + "/Figures/Loss-Accuracy.png")

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


[I 2024-04-03 07:09:46,971] A new study created in memory with name: no-name-75d32cd6-eca2-42e1-ace2-092e93442839


530


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 95.5MB/s]


Epoch 1 accuracy: 0.2344 val_accuracy: 0.375940
Epoch 2 accuracy: 0.2469 val_accuracy: 0.533835
Epoch 3 accuracy: 0.2625 val_accuracy: 0.533835
Epoch 4 accuracy: 0.2344 val_accuracy: 0.548872
Epoch 5 accuracy: 0.2750 val_accuracy: 0.503759
Epoch 6 accuracy: 0.2375 val_accuracy: 0.533835
Epoch 7 accuracy: 0.2594 val_accuracy: 0.518797
Epoch 8 accuracy: 0.2875 val_accuracy: 0.533835
Epoch 9 accuracy: 0.2812 val_accuracy: 0.563910
Epoch 10 accuracy: 0.2625 val_accuracy: 0.541353
Epoch 11 accuracy: 0.2906 val_accuracy: 0.548872
Epoch 12 accuracy: 0.2781 val_accuracy: 0.526316
Epoch 13 accuracy: 0.2656 val_accuracy: 0.533835
Epoch 14 accuracy: 0.2812 val_accuracy: 0.533835
Epoch 15 accuracy: 0.2562 val_accuracy: 0.518797
Epoch 16 accuracy: 0.2781 val_accuracy: 0.533835
Epoch 17 accuracy: 0.2844 val_accuracy: 0.541353
Epoch 18 accuracy: 0.2969 val_accuracy: 0.518797
Epoch 19 accuracy: 0.2812 val_accuracy: 0.526316
Epoch 20 accuracy: 0.2750 val_accuracy: 0.533835
Epoch 21 accuracy: 0.2781 val

[I 2024-04-03 08:32:03,997] Trial 0 finished with value: 0.5413534045219421 and parameters: {'n_layers': 2, 'n_units_l0': 270, 'dropout_l0': 0.11928042629012635, 'n_units_l1': 381, 'dropout_l1': 0.21521735195807362, 'optimizer': 'AdamW', 'lr': 1.0810830499867836e-05}. Best is trial 0 with value: 0.5413534045219421.


Epoch 60 accuracy: 0.3250 val_accuracy: 0.541353
Epoch 1 accuracy: 0.2219 val_accuracy: 0.541353
Epoch 2 accuracy: 0.2906 val_accuracy: 0.533835
Epoch 3 accuracy: 0.2844 val_accuracy: 0.533835
Epoch 4 accuracy: 0.2750 val_accuracy: 0.548872
Epoch 5 accuracy: 0.2781 val_accuracy: 0.541353
Epoch 6 accuracy: 0.2531 val_accuracy: 0.526316
Epoch 7 accuracy: 0.2375 val_accuracy: 0.541353
Epoch 8 accuracy: 0.2562 val_accuracy: 0.533835
Epoch 9 accuracy: 0.2875 val_accuracy: 0.526316
Epoch 10 accuracy: 0.2562 val_accuracy: 0.533835
Epoch 11 accuracy: 0.2937 val_accuracy: 0.541353
Epoch 12 accuracy: 0.2750 val_accuracy: 0.548872
Epoch 13 accuracy: 0.2812 val_accuracy: 0.526316
Epoch 14 accuracy: 0.3063 val_accuracy: 0.548872
Epoch 15 accuracy: 0.2531 val_accuracy: 0.563910
Epoch 16 accuracy: 0.2937 val_accuracy: 0.601504
Epoch 17 accuracy: 0.3156 val_accuracy: 0.571429
Epoch 18 accuracy: 0.3063 val_accuracy: 0.563910
Epoch 19 accuracy: 0.3125 val_accuracy: 0.556391
Epoch 20 accuracy: 0.2906 val

[I 2024-04-03 09:53:19,264] Trial 1 finished with value: 0.6240601539611816 and parameters: {'n_layers': 2, 'n_units_l0': 179, 'dropout_l0': 0.13234239853036264, 'n_units_l1': 233, 'dropout_l1': 0.19686087879438124, 'optimizer': 'SGD', 'lr': 0.03158864678248769}. Best is trial 1 with value: 0.6240601539611816.


Epoch 60 accuracy: 0.4250 val_accuracy: 0.624060
Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.6240601539611816
  Params: 
    n_layers: 2
    n_units_l0: 179
    dropout_l0: 0.13234239853036264
    n_units_l1: 233
    dropout_l1: 0.19686087879438124
    optimizer: SGD
    lr: 0.03158864678248769
Epoch 1 accuracy: 0.5377 val_accuracy: 0.541353
Epoch 2 accuracy: 0.5585 val_accuracy: 0.541353
Epoch 3 accuracy: 0.5585 val_accuracy: 0.541353
Epoch 4 accuracy: 0.5585 val_accuracy: 0.556391
Epoch 5 accuracy: 0.5698 val_accuracy: 0.601504
Epoch 6 accuracy: 0.5868 val_accuracy: 0.609023
Epoch 7 accuracy: 0.6113 val_accuracy: 0.616541
Epoch 8 accuracy: 0.6396 val_accuracy: 0.624060
Epoch 9 accuracy: 0.6434 val_accuracy: 0.609023
Epoch 10 accuracy: 0.6491 val_accuracy: 0.609023
Epoch 11 accuracy: 0.6849 val_accuracy: 0.609023
Epoch 12 accuracy: 0.7151 val_accuracy: 0.631579
Epoch 13 accuracy: 0.7170 val_ac